<span style="font-family:New York Times; font-size:1.5em; color:green;"> 
pika usage

* [https://www.rabbitmq.com/tutorials/tutorial-one-python.html](https://www.rabbitmq.com/tutorials/tutorial-one-python.html)
* [https://www.cloudamqp.com/blog/2015-05-21-part2-3-rabbitmq-for-beginners_example-and-sample-code-python.html](https://www.cloudamqp.com/blog/2015-05-21-part2-3-rabbitmq-for-beginners_example-and-sample-code-python.html)



<span style = "font-family: New York Times; font-size:1em; color:green;">
    
RabbitMQ official tutorial
* "Hello world"
* Work Queues

  each task is delivered to exactly one worker. 
* publish/subscribe

  deliver a message to multiple consumers. This pattern is known as "publish/subscribe".
* routing
* Topics
* PRC

In [ ]:
!pip install pika 

* server: `rabbitmq-server`
* client: pika
Without a server, the next step is doomed!

# Hello world

In [ ]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
# We're connected now, to a broker on the local machine 
channel = connection.channel()
# create a channel in the TCP connection.

## Publisher

###  `channel.queue_declare`


In [ ]:
channel.queue_declare(queue = 'hello') 
# create a hello queue, a recipient queue, to which the message will be delivered:

### `channel.basic_publish`

In [ ]:
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
# use a default exchange identified by an empty string
# This exchange is special ‒ it allows us to specify exactly to which queue the message should go.
print(" [x] Sent 'Hello World!'")
chanel.closed()
# The connection will be closed after the message has been published.

<span style = "font-family: New York Times; font-size:1.2em; color:green;">
The default exchange means that messages are routed to the queue with the name specified by routing_key, if it exists. (The default exchange is a direct exchange with no name).

## Receiver

### callback(ch, method, properties, body)

<span style="font-family:New York Times; font-size:1em; color:green;"> 
Receiving messages from the queue is more complex. It works by subscribing a callback function to a queue. Whenever we receive a message, this callback function is called by the Pika library.

In [ ]:
# #create a function which is called on incoming messages
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

<span style="font-family:New York Times; font-size:1em; color:green;"> 
Next, we need to tell RabbitMQ that this particular callback function should receive messages from our hello queue:

In [ ]:
channel.basic_consume(queue='hello',
                      auto_ack=True,
                      on_message_callback=callback)

<span style="font-family:New York Times; font-size:1em; color:green;"> 
In order to make sure a message is never lost, RabbitMQ supports message acknowledgments. An ack(nowledgement) is sent back by the consumer to tell RabbitMQ that a particular message had been received, processed and that RabbitMQ is free to delete it.

### `channel.start_consuming()`
<span style="font-family:New York Times; font-size:1em; color:green;"> 
And finally, we enter a never-ending loop that waits for data and runs callbacks whenever necessary.

### Temporary test

In [ ]:
class MessageQueue(object):
    def __init__(self, code, message, data):
        self.code = code
        self.message = message
        self.data = data

    def get_json(self):
        return json.dumps({'code': self.code, 'msg': self.message, 'data': self.data})

## Break down the hard parts
[RabbitMQ callback function](https://stackoverflow.com/a/34208810/7583919)

<span style = "font-family: New York Times; font-size:1em; color:green;">
    
callback(ch, method, properties, body)

## ch

"ch" is the "channel" over which the communication is happening. 

Think of a RabbitMQ connection in two parts:

* the TCP/IP connection
* channels within the connection

the actual TCP/IP connection is expensive to create, so you only want one connection per process instance. 

A channel is where the work is done with RabbitMQ. a channel exists within an connection, and you need to have the channel reference so you can ack/nack messages, etc. 

## method

i think "method" is meta information regarding the message delivery

when you want to acknowledge the message - tell RabbitMQ that you are done processing it - you need both the channel and the delivery tag. the delivery tag comes from the method parameter. 

i'm not sure why this is called "method" - perhaps it is related to the AMQP spec, where the "method" is meta-data about which AMQP method was executed?

## properties

the "properties" of the message are user-defined properties on the message. you can set any arbitrary key / value pair that you want in these properties, and possibly get things like routing key used (though this may come from "method")

properties are often uses for bits of data that your code needs to have, but aren't part of the actual message body. 

for example, if you had a re-sequencer process to make sure messages are processed in order, the "properties" would probably contain the message's sequence number.

### How the data is stored and transported?  how the data were accepted?

# Architecture of RabbitMQ

![](https://www.rabbitmq.com/img/tutorials/exchanges.png)

<span style = "font-family: New York Times; font-size:1em; color:green;">
Instead, the producer can only send messages to an exchange. An exchange is a very simple thing. On one side it receives messages from producers and the other side it pushes them to queues. The exchange must know exactly what to do with a message it receives. Should it be appended to a particular queue? Should it be appended to many queues? Or should it get discarded. The rules for that are defined by the exchange type.